In [12]:
# Takes any .reg files created in ds9 and flags the region in the corresponding FLC images' DQ arrays.

# To create the .reg files, open the FLC image in ds9 and draw a region (Region -> Shape -> Polygon) 
# around the object you want to flag (e.g. a sattelite trail), and then save this region.
# When saving the .reg file in ds9 (Region -> Save Regions), set format=ds9, coord system=image 
# and call it <dataset>_ext<1/4>.reg depending on what SCI extension the object is in, e.g. 'idj502t0q_ext_1.reg'

import glob
import os
import shutil

from astropy.io import fits
import numpy as np
import pyregion

# Make a list of all created .reg files
files = glob.glob('../flag_test/*.reg')
for f in files:
    
    # Get basename/flc name and sci extension to flag
    basename = os.path.basename(f)
    ext = int(basename.split('_')[2].split('.')[0])
    flc_filename = f.replace('_ext_{}.reg'.format(ext), '_flc.fits')
    
    # Open the relevant flc image and corresponding region file
    h = fits.open(flc_filename)
    r = pyregion.open(f).as_imagecoord(h[ext+2].header)
    
    # Make a mask using the region file, same dimensions as flc extension
    mask = r.get_mask(hdu=h[ext+2])
    
    # Make a new DQ array for this image, incorporating the region mask
    dq = h[ext+2].data
    new_dq = np.zeros(mask.shape)
    new_dq[mask == True] = 16384 # this bit is currently unused in ACS and UVIS
    new_dq += dq
    h[ext+2].data = new_dq.astype('int16')
    
    # Save this image with the updated DQ array
    h.writeto(flc_filename, overwrite=True)
    h.close()
